# Parte 4

## A)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from keras.models import Sequential
# from keras.layers import Dense, Activation
# from keras.optimizers import SGD
# from keras.utils.np_utils import to_categorical

In [2]:
def load_data():
    train = pd.read_csv('sign_mnist_train.csv')
    test = pd.read_csv('sign_mnist_test.csv')
    y_tr = train['label']
    x_tr = train.iloc[:,1:]
    y_t = test['label']
    x_t = test.iloc[:,1:]
    x_tr, x_v, y_tr, y_v = train_test_split(x_tr, y_tr, test_size=0.2, random_state=1)
    print("{0} registros de entrenamiento.".format(x_tr.shape[0]))
    print("{0} registros de validación.".format(x_v.shape[0]))
    print("{0} registros de prueba.".format(x_t.shape[0]))
    print("Cada registro posee {0} pixeles.".format(x_t.shape[1]))
    #you need to add Xval: x_v,y_v
    return(x_tr,x_v,x_t,y_tr,y_v,y_t)

In [3]:
x_tr, x_v, x_t, y_tr, y_v, y_t = load_data()

21964 registros de entrenamiento.
5491 registros de validación.
7172 registros de prueba.
Cada registro posee 784 pixeles.


In [8]:
x_tr.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
5573,112,118,125,130,136,141,143,144,148,151,...,16,138,224,219,220,221,221,221,221,221
10761,80,93,109,123,137,146,157,165,167,169,...,108,104,104,92,82,77,80,92,122,126
1604,69,0,9,49,49,56,59,62,67,71,...,76,81,80,85,82,82,75,83,81,42
10156,195,197,198,198,199,199,199,199,199,199,...,130,198,219,172,162,194,212,222,222,191
3519,238,240,243,243,244,246,248,248,248,248,...,255,255,255,255,255,255,255,255,255,255


In [4]:
def intensity_scale(X):
    X=X.apply(lambda x: x/255)
    return X

def scale_and_center(X):
    scalerX = StandardScaler()
    X = pd.DataFrame(scalerX.fit_transform(X))
    return X

In [6]:
x_tr1 = intensity_scale(x_tr)
x_v1 = intensity_scale(x_v)
x_t1 = intensity_scale(x_t)

x_tr2 = scale_and_center(x_tr)
x_v2 = scale_and_center(x_v)
x_t2 = scale_and_center(x_t)

In [ ]:

model = Sequential()
model.add(Dense(30, input_dim=x_tr.shape[1], init='uniform', activation='relu'))
model.add(Dense(30, init='uniform', activation='relu'))
model.add(Dense(25, init='uniform', activation='softmax'))
model.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_tr.values, to_categorical(y_tr), nb_epoch=100, batch_size=128, verbose=1,
validation_data=(x_v.values,to_categorical(y_v)))